In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [10]:
data = pd.read_csv("./whoop_fitness_dataset_100k.csv")
data.info()
print("Duplicates:", data.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 39 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   user_id                  100000 non-null  object 
 1   date                     100000 non-null  object 
 2   day_of_week              100000 non-null  object 
 3   age                      100000 non-null  int64  
 4   gender                   100000 non-null  object 
 5   weight_kg                100000 non-null  float64
 6   height_cm                100000 non-null  float64
 7   fitness_level            100000 non-null  object 
 8   primary_sport            100000 non-null  object 
 9   recovery_score           100000 non-null  float64
 10  day_strain               100000 non-null  float64
 11  sleep_hours              100000 non-null  float64
 12  sleep_efficiency         100000 non-null  float64
 13  sleep_performance        100000 non-null  float64
 14  light

In [11]:
data.columns = data.columns.str.lower()

In [12]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
age,100000.0,41.642950,13.257307,18.00,30.00,41.00,53.00,64.00
weight_kg,100000.0,72.707693,13.896367,38.10,62.90,72.10,82.00,115.00
height_cm,100000.0,171.754403,9.337338,149.40,165.30,171.50,178.90,195.80
recovery_score,100000.0,65.130442,17.698584,5.00,52.40,64.50,77.40,100.00
day_strain,100000.0,9.911999,4.489623,0.00,6.65,9.53,12.89,21.00
sleep_hours,100000.0,7.296272,0.972416,4.00,6.65,7.46,8.03,10.00
sleep_efficiency,100000.0,82.883422,10.222554,50.00,76.50,85.00,90.50,100.00
sleep_performance,100000.0,100.000000,0.000000,100.00,100.00,100.00,100.00,100.00
light_sleep_hours,100000.0,4.378101,0.603068,2.24,3.98,4.46,4.82,6.21
rem_sleep_hours,100000.0,1.641338,0.242734,0.81,1.48,1.66,1.82,2.43
